# Train model with different tools

## load data

In [3]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [4]:
hd_ttH=pandas.read_hdf('../../Files/skimmed/ttH.h5', 'df')
hd_ttW=pandas.read_hdf('../../Files/skimmed/ttW.h5', 'df')

In [10]:
hd_ttH.head()
hd_ttH['DRll01']

entry
0       2.833694
1       2.171907
2       2.189412
3       2.389086
4       0.472404
5       0.717594
6       3.316627
7       2.160175
8       2.056099
9       0.709195
10      1.692138
11      3.211834
12      1.691607
13      1.851815
14      0.306701
15      0.795250
16      1.963612
17      0.613543
18      2.307259
19      0.852842
20      1.176668
21      2.964810
22      1.904576
23      2.985362
24      0.826391
25      1.260774
26      2.499718
27      2.962825
28      2.518363
29      3.475383
          ...   
9970    1.076918
9971    1.883128
9972    1.285473
9973    2.013553
9974    1.226869
9975    3.194727
9976    1.009742
9977    0.983298
9978    3.314262
9979    2.080025
9980    2.389684
9981    2.177511
9982    2.372535
9983    2.513709
9984    2.304936
9985    2.047837
9986    3.415068
9987    0.558144
9988    1.899859
9989    1.493657
9990    1.633292
9991    0.754516
9992    3.664529
9993    2.138528
9994    1.844030
9995    1.857249
9996    3.210032
9997    

 * X = signal + background data;
 * object y - hold the class labels.

The correspondence between `X` and `y` is given by the index

In [6]:
X = np.concatenate((hd_ttH,hd_ttW)) # training data                                                                                                                               
y = np.concatenate((np.ones(hd_ttH.shape[0]),np.zeros(hd_ttW.shape[0]))) # class lables                                                                                           


TypeError: slice indices must be integers or None or have an __index__ method